In [4]:
import os
import re
import requests
from itertools import chain
import networkx as nx
from simplejson import loads, load,dumps,dump
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import uuid
import gc

In [2]:
concept_by_id_list_api = "http://snomed-20160731-release-server-2016-10-10.slade360emr.com/terminology/concept_list_by_id/"

headers = {'Expect': '100-Continue'}
def get_concept_term_by_id_list(id_list):
    """Takes a list of valid SCTIDs and returns a dict mapping 
    {'<sctid>': <preferred_term>, '<sctid>': <preferred_term>, ...}"""
    ids = ','.join(id_list)
    try:
        response = requests.post(concept_by_id_list_api,data={'sctid_list': ids}).text
    except Exception as e:
        print("Request failed at id {}. Retrying...".format(id_list))
        return get_concept_term_by_id_list(id_list)
    try:
        data = loads(response)
    except Exception as e:
        print(response)
    return data

In [6]:
ADJACENCY_LIST_FILE='transitive_closure_adjacency_list.adjlist'
if not os.path.isfile(ADJACENCY_LIST_FILE):
    response=requests.get(
       "http://snomed-20160731-release-server-2016-10-10.slade360emr.com/terminology/relationships/transitive_closure/adjacency_list"
    )
    with open(ADJACENCY_LIST_FILE, 'w') as file:
        file.write(response.text)

In [11]:
# Initialize an empty directed graph (digraph) and use 
# read_adjlist to read the downloaded file using the empty digraph.

G=nx.DiGraph()
DG = nx.read_adjlist(ADJACENCY_LIST_FILE, create_using=G)

# get the ancestry graph of a single concept. This is where you'll enter the concept id that you want to investigate
drug = '211000001101'

def get_ancestry_graph(drug):
    drug_ancestors = nx.ancestors(DG, drug)
    ancestry_subgraph = DG.subgraph(drug_ancestors)
    return ancestry_subgraph

H = get_ancestry_graph(drug)
# clear memory
del DG
gc.collect()

0

In [12]:
from networkx.drawing.nx_agraph import to_agraph
from networkx.readwrite import json_graph
from simplejson import dumps
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_pydot import pydot_layout
from matplotlib import pylab

def render_reversed_ancestry_graph(graph, file_name):
    """Reverses the ancestry graph of a SNOMED CT concept, and renders it as a SVG image."""
    leaf_nodes = [x for x in graph.nodes_iter() if graph.out_degree(x)==0]
    graph.add_node(drug)
    for node in leaf_nodes:
        graph.add_edge(node, drug)
    
    reverse_graph = graph.reverse(copy=True)
    
    plt.figure(num=None, figsize=(30, 30), dpi=160)
    plt.axis('off')
    fig = plt.figure(1)
    pos = pydot_layout(reverse_graph, prog='dot', format='svg')
    node_names = get_concept_term_by_id_list(reverse_graph.nodes())
    labeled_nodes = dict(zip(reverse_graph.nodes(), 
                             list(map(lambda x: node_names[x] + ' | {} |'.format(x), 
                                      reverse_graph.nodes()))))
    
    nx.draw_networkx_nodes(reverse_graph,pos)
    nx.draw_networkx_edges(reverse_graph,pos)
    
    nx.draw_networkx_labels(reverse_graph,pos, labels=labeled_nodes)

    cut = 1.2
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig

render_reversed_ancestry_graph(H, 'drug-sample-{}.svg'.format(uuid.uuid4()))